# Data Ingestion Module

In [1]:
import os
!pwd

/teamspace/studios/this_studio/text-summarizer/research


In [2]:
os.chdir('../')
!pwd

/teamspace/studios/this_studio/text-summarizer


### Basic Configuration

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_uri: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

### Configuration updates

In [12]:
from typing import Optional
class CofigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])
    
    def get_data_ingestion_config(self) -> Optional[DataIngestionConfig]:
        try:
            config = self.config.data_ingestion
            
            # Ensure that the root directory for data ingestion exists
            create_directories([config.root_dir])
            
            return DataIngestionConfig(
                root_dir=config.root_dir,
                source_uri=config.source_uri,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
            )
        except AttributeError as e:
            logger.error(f"Missing data ingestion configuration: {e}")
            return None
        except Exception as e:
            logger.error(f"Error in creating DataIngestionConfig: {e}")
            return None

In [13]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import create_logger

### Components

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.logger = create_logger(logger_name=__name__)
        
    def download_data(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                self.logger.info(f"Crating the directory {self.config.local_data_file}")
                os.makedirs(self.config.local_data_file)
                
                filename, headers = request.urlretrieve(
                url=self.config.source_uri, 
                filename=self.config.local_data_file)
            else:
                self.logger.info(f"Directory {self.config.local_data_file} already exists")
            self.logger.info(f"Downloading data from {self.config.source_uri}")
        except Exception as e:
            self.logger.error(f"Error downloading data from {self.config.source_uri}")
            self.logger.error(e)
            raise e
    
    def extract_zip_file(self):
        try:
            os.makedirs(self.config.unzip_dir, exist_ok=True)
            
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)
            self.logger.info(f"Extracting zip file to {self.config.unzip_dir}")
        except Exception as e:
            self.logger.error(f"Error extracting zip file to {self.config.unzip_dir}")
            self.logger.error(e)
            raise e

In [15]:
config = CofigurationManager()

data_ingestion_config = config.get_data_ingestion_config()

data_ingestion = DataIngestion(data_ingestion_config)

dataclass.download_data()
dataclass.extract_zip_file()

TypeError: Subscripted generics cannot be used with class and instance checks